In [3]:
import sys
sys.path.append("..")

In [4]:
from low_precision_utils import quant

/home/tl2020/convergence_srsgd/.venv/lib64/python3.9/site-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [5]:
import torch

In [6]:
x = torch.randn((30,)) * 100


In [7]:
x1 = quant.IntQuant(fl=-4).quant(x)

In [8]:
x1

tensor([ -32.,   16., -240., -192.,   -0.,   64.,   48.,  -64.,  -16.,  112.,
         256., -256.,  -32., -240.,  -64.,   16.,  -80.,   48., -176.,   64.,
        -112.,  176.,   64.,   16.,   80., -112., -160.,   16.,   16.,  -16.])

In [9]:
x

tensor([ -41.5988,   24.4981, -247.1406, -177.6817,   -7.8955,   58.1012,
          48.5418,  -60.0989,  -19.2383,  112.2188,  253.3214, -258.6229,
         -29.0268, -246.2362,  -73.4833,   11.2230,  -87.8874,   50.1658,
        -184.5584,   57.0788, -108.9716,  182.3041,   66.1753,    5.1976,
          78.3296, -111.4436, -152.7802,   13.0764,   16.6714,   -4.4092])

In [10]:
x2 = quant.NoiseQuant(std=0.1).quant(x)

In [11]:
x2

tensor([ -41.6874,   24.2796, -247.2363, -177.7977,   -8.0358,   58.0377,
          48.6358,  -60.0159,  -19.3872,  112.1782,  253.4795, -258.6210,
         -29.0780, -246.2655,  -73.5209,   11.3378,  -87.9255,   50.0567,
        -184.7548,   57.1682, -109.1733,  182.3753,   65.9985,    5.2073,
          78.4330, -111.6468, -152.8599,   13.0707,   16.6053,   -4.5312])

In [12]:
x.sum()

tensor(-834.1700)

In [13]:
x1.sum()

tensor(-800.)

In [14]:
x2.sum()

tensor(-835.6736)

In [15]:
def estimate_variance(length, repeats=100):
    e1 = torch.zeros((repeats,))
    e2 = torch.zeros((repeats,))

    for i in range(repeats):
        x = torch.randn((length,)) + 100
        x1 = quant.IntQuant(fl=-8).quant(x)
        x2 = quant.NoiseQuant(std=0.5).quant(x)
        e1[i] = (x1.sum() - x.sum()) / length
        e2[i] = (x2.sum() - x.sum()) / length
    print(x1[:10])

    return e1.var(), e2.var()

In [16]:
for length in [100, 200, 400, 800, 1600, 3200]:
    print(estimate_variance(length, 200))

tensor([  0.,   0.,   0., 256.,   0.,   0., 256.,   0.,   0.,   0.])
(tensor(143.0426), tensor(0.0028))
tensor([  0.,   0.,  -0.,   0., 256.,   0.,   0.,  -0.,  -0.,   0.])
(tensor(82.4569), tensor(0.0012))
tensor([  0., 256.,   0.,   0.,   0.,   0., 256.,   0.,   0.,   0.])
(tensor(39.0596), tensor(0.0006))
tensor([256., 256., 256., 256.,  -0., 256.,   0., 256.,   0., 256.])
(tensor(16.3998), tensor(0.0003))
tensor([  0.,   0.,   0., 256.,   0.,   0.,   0.,   0., 256.,   0.])
(tensor(9.4365), tensor(0.0002))
tensor([  0.,   0.,   0., 256., 256.,   0.,   0., 256.,   0., 256.])
(tensor(4.9419), tensor(8.3436e-05))


In [28]:
ls = [torch.randn((100,)).sum() / 100 for _ in range(100)]

In [29]:
torch.var(torch.tensor(ls))

tensor(0.0106)